In [1]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime
import json
from pandas.io.json import json_normalize
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
#import nltk package
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from nltk.tokenize import word_tokenize, RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#Written by Isil and Lau

In [2]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

#input date in the formate 'dd-mm-yy'
def get_timestamp(date):
    tmp = date.split('-')
    dt_obj = dt.datetime(int(tmp[2]),int(tmp[1]),int(tmp[0]))
    return int(time.mktime(dt_obj.timetuple()))

#Sentiment calculation
def sentiment_calc(text):
    try:
        return sia.polarity_scores(text)['compound']
    except:
        return None
    
# apply sentiment calculation to headlines
def process_text(headlines):
    tokens = []
    for line in headlines:
        toks = tokenizer.tokenize(line)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    
    return tokens

In [3]:
# input parameters, input date in the formate 'dd-mm-yy'
size = "1000"
start_date = "17-08-2019"
end_date = "28-08-2019"
keyword = "cardano"

# base url and generate timestamp
base = "https://api.pushshift.io/reddit/search/submission/?"
st_date = str(get_timestamp(start_date))
en_date = str(get_timestamp(end_date))

In [4]:
#Sentiment distribution
#Divided into positive, negative and neutral sentiment
#query data
url = base + "size="+size+"&after="+ st_date + "&before=" + en_date + "&subreddit=" + keyword + "&sort_type=score&sort=desc"
response = requests.get(url)
data = response.json()

#generate dataframe
tmp = json_normalize(data, 'data')
#cols=['selftext', 'title', 'score', 'num_comments', 'created_utc', 'full_link']
cols=['title', 'score', 'created_utc']
df2 = pd.DataFrame(tmp, columns=cols)
pd.set_option('display.max_colwidth', -1)

#add date
_timestamp = df2["created_utc"].apply(get_date)
df2 = df2.assign(timestamp = _timestamp)
df2['Date'] = pd.to_datetime(df2['timestamp'])
tmp = [i.strftime('%m/%Y') for i in df2['Date']]
df2['Month'] = pd.Series(tmp)

#Sentiment calculation
df2['sentiment'] = df2['title'].apply(sentiment_calc)

#Add sentiment label, +1 for positive and -1 for negative, 0 for neutral
df2['label'] = 0
df2.loc[df2['sentiment'] > 0.2, 'label'] = 1
df2.loc[df2['sentiment'] < -0.2, 'label'] = -1

#Generate csv for the query data
df = df2.drop(columns = ['timestamp'])
df_sim = df.drop(columns = 'title')
file_name = 'Reddit_data' + '_' + keyword + '_' + start_date + '.csv'
simp = 'Simp_Reddit_data' + '_' + keyword + '_' + start_date + '.csv'
df.to_csv(file_name, index=True)
df_sim.to_csv(simp, index=True)

#Distribution in python, I shall do this wit sql instead
#print(df2.label.value_counts())
#print(df2.label.value_counts(normalize=True) * 100)

In [7]:
# obtain top keywords from each sentiment. Can take parameters other than "most_common"

#Positive headlines. For negative change 1 to -1
pos_lines = list(df[df.label == 1].title)
pos_tokens = process_text(pos_lines)
pos_freq = nltk.FreqDist(pos_tokens)

#generate pos_keyword_df
tmp = pd.DataFrame({'col':pos_freq.most_common(20)})
tmp[['keyword','frequency']] = pd.DataFrame(tmp.col.values.tolist(), index= tmp.index)
pos_df = tmp.drop(columns = 'col')

#Manuel specification on grouping the full list
full_list = list(pos_df['keyword'])
ada = ['cardano', 'ada']
market = ['crypto', '02btc', 'exchange', 'bitcoin']
car_inst = ['charles','hoskinson','emurgo','iohk']
car_apps = ['gambling', 'cryptoroulette','free']
car_tech = ['platform', 'stake', 'blockchain', 'progress','project', 'marlowe', 'plutus', 'contracts']
oth_proj = []
not_others = ada + market + car_apps + car_tech
others = [i for i in list(pos_df['keyword']) if i not in not_others] 

#info_array = [ada, market, car_apps, car_tech, others]
info_array = [ada, market, car_apps, car_inst, car_tech, oth_proj, others]
str_info_array = ['cardano', 'market', 'cardano Dapps', 'cardano institutions', 'cardano technology', 'other blockchain projects', 'other']
#frequency count in each group
freq_array = [0 for i in range(0,len(info_array))]
for j in range(0,len(info_array)):
    freq_array[j] = sum([pos_df['frequency'][i] for i in range(0,len(full_list)) if pos_df['keyword'][i] in info_array[j]])

pos_grp_freq = pd.DataFrame(freq_array, index = str_info_array, columns = ['frequency'])

#naming
tot_pos_freq_name = 'Reddit_pos' + '_' + keyword + '_' + start_date + '.csv'
pos_freq_name = 'Reddit_pos_grouped' + '_' + keyword + '_' + start_date + '.csv'

pos_df.to_csv(tot_pos_freq_name, index=False) 
pos_grp_freq.to_csv(pos_freq_name)

#Positive headlines. For negative change 1 to -1
neg_lines = list(df[df.label == -1].title)
neg_tokens = process_text(neg_lines)
neg_freq = nltk.FreqDist(neg_tokens)

#generate neg_keyword_df
tmp = pd.DataFrame({'col':neg_freq.most_common(20)})
tmp[['keyword','frequency']] = pd.DataFrame(tmp.col.values.tolist(), index= tmp.index)
neg_df = tmp.drop(columns = 'col')

#Manuel specification on grouping the full list
full_list = list(neg_df['keyword'])
ada = ['cardano', 'ada']
market = ['crypto', '02btc','market','holdings']
car_apps = []
car_inst = ['emurgo','charles','hoskinson','iohk']
car_tech = ['platform','development','project','protocol','contract']
oth_proj = ['facebook','consensys','scam','block','craig','eos']
not_others = ada + market + car_tech + car_inst
others = [i for i in list(neg_df['keyword']) if i not in not_others] 

info_array = [ada, market, car_apps, car_inst, car_tech, oth_proj, others]
str_info_array = ['cardano', 'market', 'cardano Dapps', 'cardano institutions', 'cardano technology', 'other blockchain projects', 'other']
#frequency count in each group
freq_array = [0 for i in range(0,len(info_array))]
for j in range(0,len(info_array)):
    freq_array[j] = sum([neg_df['frequency'][i] for i in range(0,len(full_list)) if neg_df['keyword'][i] in info_array[j]])

neg_grp_freq = pd.DataFrame(freq_array, index = str_info_array, columns = ['frequency'])

#naming
tot_neg_freq_name = 'Reddit_neg' + '_' + keyword + '_' + start_date + '.csv'
neg_freq_name = 'Reddit_neg_grouped' + '_' + keyword + '_' + start_date + '.csv'

neg_df.to_csv(tot_neg_freq_name, index=False) 
neg_grp_freq.to_csv(neg_freq_name)